In [43]:
%matplotlib inline

In [2]:
import sys, os
from pathlib import Path
import pathlib
streetview_path = Path(os.path.abspath('')).parent.absolute()

if (not (streetview_path in sys.path)) :
    sys.path.append(str(streetview_path))

In [3]:
from xml.dom import minidom
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import numpy as np

import streetview

### Overpass

In [16]:
import overpy
import time
import folium
import numpy as np
import geopandas as gpd

api = overpy.Overpass()

# Effiel Tower (France)
latitude = 48.8583701
longitude = 2.2944813

# Hoan Kiem Lake (Vietname)
# lat = 21.01931
# lng = 105.830341

radius = 200

In [17]:
# https://wiki.openstreetmap.org/wiki/Key:highway#Roads

highway_allow = ["motorway","trunk","primary","secondary","tertiary","unclassified",
                 "residential"," motorway_link","trunk_link","primary_link",
                 "secondary_link","tertiary_link"]
allow_list = "|".join(highway_allow)

In [18]:
# Create a map centered on the specified latitude and longitude
m = folium.Map(location=[latitude, longitude], zoom_start=14)

# Query the Overpass API for ways within the specified radius
api = overpy.Overpass()
# result = api.query(f"""
#     way(around:{radius},{latitude},{longitude})[highway];
#     out;
# """)
result = api.query(f"""
    way(around:{radius},{latitude},{longitude})[highway~"^({allow_list})$"];
    out;
""")

In [19]:
waypoint = set()

for way in result.get_ways():
    while True:
        try:
            
            polyline = list()
            
            nodes = way.get_nodes(resolve_missing=True)
            for node in way.nodes:
                polyline.append( (node.lat, node.lon) )
                waypoint.add( (node.lat, node.lon) )
            
            folium.PolyLine(polyline).add_to(m)
        except OverpassTooManyRequests:
            time.sleep(1)
            print('Retrying...')
            continue
        except OverpassGatewayTimeout:
            time.sleep(10)
            print('OverpassGatewayTimeout, retrying...')
            continue
        break

In [20]:
m

In [ ]:
def expand_lat_lon(lat_lon: tuple, lat_lon_before: tuple) -> list:
    
    """Expand latitude longitude to get more images between two points"""
    
    lat, lon = lat_lon
    lat_before, lon_before = lat_lon_before
    diff_lat = lat - lat_before
    diff_lon = lon - lon_before
    step_size = 3e-5
    steps_lat = abs(diff_lat) / 1e-5
    steps_lon = abs(diff_lon) / 1e-5
    factor_lat = step_size if diff_lat > 0 else -step_size
    factor_lon = step_size if diff_lon > 0 else -step_size
    jumps = round(max([steps_lat, steps_lon]) / min([steps_lat, steps_lon]))
    expanded = []

    for i in range(int(max([steps_lat, steps_lon]))):

        if not (i % jumps):
            # Find the smaller one
            if steps_lat <= steps_lon:
                lat += factor_lat
            else:
                lon += factor_lon

        # Find the bigger one
        if steps_lat >= steps_lon:
            lat += factor_lat
        else:
            lon += factor_lon
        
        expanded.append((lat, lon))
    
    return expanded


def download_images(coords: list, 
                    show_img: bool=True, 
                    img_dir: str='', 
                    video_dir: str='') -> None:
    """Function to download panorama images from coordinates list"""
    if img_dir:
        img_cnt = 0
        check_path(img_dir)
        
    if video_dir:
        check_path(video_dir)
    
    lat_before = None
    lon_before = None
    pan_before = None
    
    for lat, lon in coords:
        
        if lat_before is not None:
            expanded_lat_lon = expand_lat_lon((lat, lon), (lat_before, lon_before))
        
        if 'expanded_lat_lon' in locals():
            for e_lat, e_lon in expanded_lat_lon:
                print(f"Searching for lat={e_lat}, lon={e_lon}")
                panoids = streetview.panoids(lat=e_lat, lon=e_lon)
                try:
                    panoid = panoids[0]['panoid']
                except IndexError:
                    continue
                panorama = streetview.download_panorama_v3(panoid, zoom=3, disp=False)
                # Don't save the same image twice
                if pan_before is not None:
                    if np.array_equal(panorama, pan_before):
                        continue
                        
                pan_before = panorama
                
                if show_img:
                    plt.figure(figsize=(15, 15))
                    plt.imshow(panorama)
                    plt.axis('off')
                    plt.show()

                # Save img
                if img_dir:
                    # file name format: img_number_latitude_longitude.png
                    plt.imsave(f"{img_dir}/img_{img_cnt}_{lat}_{lon}.jpg", panorama)
                    img_cnt += 1
        
        # Skip first frame
        lat_before = lat
        lon_before = lon
        
    if 'out' in locals():
        out.release()

In [ ]:
coords = get_lat_long(gpx_path)

In [ ]:
download_images(coords, show_img=True, img_dir='./images/', video_dir='./video/')

In [ ]:
panoids = streetview.panoids(lat=40.75388056, lon=-73.99697222)
panoid = panoids[0]['panoid']
panorama = streetview.download_panorama_v3(panoid, zoom=2, disp=False)
plt.imshow(panorama)

In [ ]:
panoid = "ChAzMTM1YWI3NzhlYTEwYmI3"
panorama = streetview.download_panorama_v3(panoid, zoom=2, disp=False)
panorama

### Refacor sv-dlp

In [25]:
from PIL import Image

In [27]:
metadata = None

In [42]:
class MetadataStructure:
    dict_instance = list()
    
    def __init__(self, service=None, pano_id=None, lat=None, lng=None, date=None, size=None, max_zoom=None, misc={}, timeline={}, linked_panos={}):
        self.pano_id = pano_id
        self.lat = lat
        self.lng = lng
        self.date = date
        self.size = size
        self.max_zoom = max_zoom
        self.timeline = timeline
        self.dict_instance.append(self)
        
    def __repr__(self):
        return (
                f"{self.__class__.__name__}("
                f"pano_id={self.pano_id}, "
                f"lat={self.lat}, "
                f"lng={self.lng}, "
                f"date={self.date}, "
                f"size={self.size}, "
                f"max_zoom={self.max_zoom}, "
                f"timeline={self.timeline}, "
        )
        
    @classmethod
    def dict(cls):
        for instance in cls.dict_instance:
            return instance.__dict__

In [44]:
def get_metadata(pano_id=None, lat=None, lng=None, get_linked_panos=False) -> MetadataStructure:
        """
        Calls allocated service's `get_metadata()` function to obtain metadata
        with given input, and store it to class and variable.
        
        Metadata is returned in a `MetadataStructure` object, providing
        the developer a more structured and organized way to handle 
        metadata information with various attributes.
        Additionally, the `.dict()` method returns the attributes of 
        each instance of the MetadataStructure class in the form of a 
        dictionary, allowing for easy access and manipulation of the 
        metadata information.
        
        sv_dlp's metadata structure is designed with compatibility in mind, 
        allowing developers to tinker with it no matter the service picked. 
        An example of the returned metadata is the one below:
        ```python
        metadata = MetadataStructure(
            service=service, 
            pano_id=pano_id, 
            lat=lat, 
            lng=lng, 
            date=datetime.datetime(), 
            size=image_size, 
            max_zoom=max_zoom, 
            timeline=[{'pano_id': 'pano_id', 'date': datetime.datetime()}], 
            linked_panos={{'pano_id': pano_id, 'lat': lat, 'lng': lng, 'date': datetime.datetime()}}, 
            misc={}
        )
        ```
        
        Additionally, the developer has the option to access the metadata 
        in dictionary form by calling the `.dict()` method. An example is:
        ```python
        metadata = {
            "service": service,
            "pano_id": pano_id,
            "lat": lat,
            "lng": lng,
            "date": datetime.datetime(),
            "size": image_size,
            "max_zoom": max_zoom,
            "misc": { # Only use with exclusive service features
                "is_trekker": len(json[1][0][5][0][3][0][0][2]) > 3,
                "gen": gen,
            },
            "timeline": {
                [{'pano_id'}: pano_id, "date": date}],
                [{'pano_id'}: pano_id, "date": date}],
                [{'pano_id'}: pano_id, "date": date}],
                # and so on...
            }
            "linked_panos": {
                [{'pano_id'}: pano_id, "date": date, "lat": lat, "lng" lng}],
                [{'pano_id'}: pano_id, "date": date, "lat": lat, "lng" lng}],
                [{'pano_id'}: pano_id, "date": date, "lat": lat, "lng" lng}],
                # and so on... 
                # only added if get_linked_panos is true
            },
        }
        ```
        
        Parameters
        ----------
        str:    pano_id
            Panorama ID - Might not work with some services
        float:  lat
            Latitude
        float:  lng
            Longitude
        bool:   get_linked_panos
            Sets if linked panos should be returned or not

        Returns
        ----------
        MetadataStructure:  metadata
            Metadata of given input
        MetadataStructure:  self.metadata
            Stores metadata in class
        func:               .dict()
            Function inside object that translates `MetadataStructure` object to dictionary
        """
        md = get_metadata(pano_id=pano_id, lat=lat, lng=lng)
        self.metadata = md
        return md

In [48]:
def _build_tile_arr(metadata, zoom=2):

    pano_id = metadata.pano_id
    
    if zoom == 0: # zoom 0 is an uncropped panorama preview
        arr = [[urls._build_tile_url(pano_id=pano_id, zoom=zoom, x=0, y=0)]]
    else:
        x_axis = metadata.size[0][zoom][0][1] // metadata.size[1][1]
        y_axis = metadata.size[0][zoom][0][0] // metadata.size[1][0]
        arr = [[] for _ in range(y_axis)]
        for y in range(y_axis):
            print(y)
            for x in range(x_axis):
                url = urls._build_tile_url(pano_id=pano_id, zoom=zoom, x=x, y=y)
                arr[y].append(url)
    print(arr)
    return arr

In [51]:
def download_panorama(pano_id=None, lat=None, lng=None, zoom=3) -> Image:
        """
        Obtains Tile URLs List from a given Panorama ID/Coordinate with a
        specified zoom, downloads each row in a multithreaded way and stitches them.
        
        If self.metadata is not allocated (or does not match with given input),
        `get_metadata` is automatically called.
        
        Parameters
        ----------
        str:    pano_id
            Panorama ID - Might not work with some services
        float:  lat
            Latitude
        float:  lng
            Longitude

        Returns
        ----------
        Image:  img
            Stitched Panorama Image in PIL.Image format
        Image:  tile_imgs
            List of Tile Images where each element is stored
            in a PIL.Image format
        Tuple:  tuple
            A tuple is returned if only one
            variable is assigned
        """

        if metadata == None:
            print(f"Obtainin Metadata...")
            if pano_id != None:
                get_metadata(pano_id=pano_id)
            elif lat and lng != None:
                get_metadata(lat=lat, lng=lng)

        if zoom == -1:
            zoom = self.metadata.max_zoom / 2
            zoom = round(zoom + .1)

        print(f"[{self.service_str}]: Building Tile URLs...")
        tile_arr = streetview.urls._build_tile_arr(self.metadata, zoom)
        img, tiles_imgs = download.panorama(tile_arr, self.metadata)
        return img, tiles_imgs


In [54]:
pano = "fzJzOcJLZPq-_QPBJzl5Dg"
lat=6.603079535799973
lng=-73.99819681137278
print(download_panorama(pano))

Obtainin Metadata...


RecursionError: maximum recursion depth exceeded

In [37]:
size=[[[[256, 512]], [[512, 1024]], [[1024, 2048]], [[2048, 4096]], [[4096, 8192]], [[8192, 16384]]], [512, 512]]
size[0]

[[[256, 512]],
 [[512, 1024]],
 [[1024, 2048]],
 [[2048, 4096]],
 [[4096, 8192]],
 [[8192, 16384]]]

In [39]:
size[0][3][0][0]

2048

In [40]:
size[1][0]

512

In [41]:
2048 // 512

4

In [ ]:
download_panorama(lat=6.603079535799973, lng=-73.99819681137278)